In [ ]:
import numpy as np
import torch

In [ ]:
def convol(img,filter):
    """It takes image and filter and performed convolution operation
    
    """
    sum=0
    row,cols=img.shape
    for i in range(row):
        for j in range(cols):
            sum+=img[i][j]*filter[i][j]
    return sum        

In [ ]:
def conv2D(img,filter,b):
    
    # if (img.shape[0]!=img.shape[1]):
    #     return 0
    s=img.shape[0]
    f=filter.shape[0]
    size=s-f+1
    feature_map=torch.zeros((size,size))
    for i in range(size):
        for j in range(size):
            feature_map[i][j]=convol(img[i:i+3,j:j+3],filter)+b
    return feature_map


Importing Dataset and Vaidating Functions created Above

In [ ]:
from tensorflow import keras
from keras.datasets import mnist

In [ ]:
(train_images, train_labels), (test_images, test_labels) =mnist.load_data()

In [ ]:
filter=np.array([[-1,0,1,0],[-1,0,1,0],[-1,0,1,0]])

In [ ]:
feature_map=conv2D(train_images[0],filter,2)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(feature_map,cmap="gray")

In [ ]:
def flatten(X):
    return X.reshape(1, -1)


In [ ]:
       
def Maxpooling(feature_map, pool_size=2, strides=2):
    H, W = feature_map.shape
    h = (H - pool_size)//strides + 1
    w = (W - pool_size)//strides + 1
    pool = torch.zeros((h, w), dtype=feature_map.dtype)

    for i in range(h):
        for j in range(w):
            window = feature_map[i*strides:i*strides+pool_size, j*strides:j*strides+pool_size]
            max_val = torch.max(window)
            pool[i, j] = max_val

    return pool

In [ ]:
def Avgpooling(feature_map, pool_size=2, strides=2):
    H, W = feature_map.shape
    h = (H - pool_size)//strides + 1
    w = (W - pool_size)//strides + 1
    pool=np.zeros((h,w))
    for i in range(h):
        for j in range(w):
            window=feature_map[i*strides:i*strides+pool_size,j*strides:j*strides+pool_size]
            _1d=np.reshape(window,(1,-1))
            max_val=np.mean(_1d)
            pool[i][j]=max_val
    return pool        


In [ ]:
pl=Maxpooling(feature_map)

In [ ]:
plt.imshow(pl,cmap="gray")

In [ ]:
def tanh(x):
    return torch.tanh(x)

In [ ]:
import torch

def softmax(x):
    if x.ndim == 1:
        exps = torch.exp(x - torch.max(x)) 
        return exps / torch.sum(exps)           
    elif x.ndim == 2:
        exps = torch.exp(x - torch.max(x, dim=1, keepdim=True)[0])
        return exps / torch.sum(exps, dim=1, keepdim=True)


In [ ]:
def feed_forward(arr, w1, b1, w2, b2):
    pre_act = w1.T @ arr + b1      
    act = tanh(pre_act)
    pre_act_output =  w2.T @ act + b2
    return softmax(pre_act_output)

In [ ]:

def sparse_categorical_cross_entropy(y_true, y_pred, epsilon=1e-12):
    y_pred = torch.clamp(y_pred, min=epsilon, max=1.0 - epsilon)
    loss = -torch.log(y_pred[y_true])
    return loss



In [ ]:

def get_flatten(img, b):
    Image = img / 255.0
    filter = torch.tensor([[-1,0,1,0],[-1,0,1,0],[-1,0,1,0]], dtype=torch.float32)

    # Convolution + pooling
    fm  = conv2D(Image, filter, b)
    pool = Maxpooling(fm)
    fm2 = conv2D(pool, filter, b)
    pre_flat = Maxpooling(fm2)

    fcc_input = flatten(pre_flat)
    fcc_input = fcc_input.T  

    return fcc_input


In [ ]:
# feed_forward(fcc_input,w1,b1,w2,b2)

In [ ]:
test_images.shape[0]

In [ ]:
def gradient_descent_cnn(img_train, img_label, epochs, lr=0.001):
    # Initialize weights and biases properly
    w1 = torch.randn(25, 68, dtype=torch.float32, requires_grad=True)
    w2 = torch.randn(68, 10, dtype=torch.float32, requires_grad=True)
    
    b1 = torch.zeros(68, 1, dtype=torch.float32, requires_grad=True)
    b2 = torch.zeros(10, 1, dtype=torch.float32, requires_grad=True)
    b  = torch.tensor(2.0, dtype=torch.float32, requires_grad=True)

    # Scale weights safely without breaking requires_grad
    with torch.no_grad():
        w1 *= 0.01
        w2 *= 0.01

    for epoch in range(epochs):
        total_loss = 0.0
        for i in range(img_train.shape[0]):
            img = img_train[i]

            w1.grad = None
            w2.grad = None
            b1.grad = None
            b2.grad = None
            b.grad = None

            fcc_in = get_flatten(img, b)
            pred   = feed_forward(fcc_in, w1, b1, w2, b2) 

            loss = sparse_categorical_cross_entropy(img_label[i], pred)
            total_loss += loss.item()

            loss.backward()

            with torch.no_grad():
                w1 -= lr * w1.grad
                w2 -= lr * w2.grad
                b1 -= lr * b1.grad
                b2 -= lr * b2.grad
                b  -= lr * b.grad

        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/img_train.shape[0]:.4f}")

    return w1, b1, w2, b2, b



In [ ]:
w1, b1, w2, b2, b=gradient_descent_cnn(train_images[0],train_labels[0],20)

In [ ]:
train_images.shape